<a href="https://colab.research.google.com/github/shashank-r97/NLP/blob/master/Style/Model_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.6/dist-packages (0.6.2)


In [3]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases

Using TensorFlow backend.


In [4]:
%%time
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
df = df[df['attribute_name'] == 'style'].drop(columns = ['attribute_name'])
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'businesscasual' if x == 'business casual' else x)

CPU times: user 375 ms, sys: 58.7 ms, total: 434 ms
Wall time: 472 ms


In [5]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value,details
0,01DPGV4YRP3Z8J85DASGZ1Y99W,frame,les second medium noir,"minimal , modern styling meet refined luxury l...",accessory,casual,NaN
5,01DPH1DEN9G2WM7WAMJMD0A9W4,j crew,tie waist shirtdress stripe,take classic button silhouette turn ultra flat...,dressesandjumpsuits,casual,NaN
12,01E2KYW52BAG606GQ7A9H5R0KD,alo,interval microfleece pullover hoodie,articulate seam extra wide rib hem create shap...,unknown,casual,"xs 0 2 , s 4 6 , m 8 10 , l 12 14"
13,01DT513RRYT3SKH6X25G5VCH6B,chlo,leather ankle boot,heel measure approximately 55 mm 2 inch 30 mm ...,shoe boots ankle,androgynous,"fit small size , size large normal available s..."
20,01E2KM0KW6NB1JKMZVRXR6H8G2,alo,stadium quarter zip hoodie,supersoft hoodie design elastic hem cuff perfe...,unknown,casual,"xs 0 2 , s 4 6 , m 8 10 , l 12 14"


In [0]:
df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category'] + ' ' + df['details']).apply(str)

In [0]:
X = df['text'].values
y = pd.get_dummies(df['attribute_value'])
label_list = y.columns
y = y.values

In [0]:
def encode_1gram(X, mode = 'binary'):
    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X, mode)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [0]:
def encode_2gram(X, mode='binary'):
    phrases = Phrases(X, min_count=30)
    bigrams = Phraser(phrases)
    X = list(bigrams[X])

    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X, mode)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [0]:
#Tokenizer.texts_to_matrix()

In [0]:
X1, length1, vocab_size1 = encode_1gram(X, mode = 'tfidf')

In [0]:
X2, length2, vocab_size2 = encode_2gram(X, mode = 'tfidf')

In [0]:
#X1 = X1.reshape(-1, 155, 1)
#X2 = X2.reshape(-1, 155, 1)

In [14]:
X1.shape

(10887, 199)

In [0]:
X = np.concatenate([X1, X2], axis = 1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [0]:
def define_model():
    inputs = Input(shape=(length1+length2,))
    #x1 = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(inputs1)
    embedding = Embedding(input_dim=vocab_size1, output_dim=100)(inputs)

    #inputs2 = Input(shape=(length2,))
    #x2 = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(inputs2)
    #dense2 = Dense(100, activation='relu')(inputs2)
    #embedding2 = Embedding(input_dim=vocab_size2, output_dim=100)(inputs2)

    #merged = concatenate([inputs1, inputs1])
    x = LSTM(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.15)(embedding)

    #x = Dense(100, activation = 'relu')(inputs)


    x = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(x)
    x = Conv1D(filters=num_classes, kernel_size=length1+length2, padding='valid')(x)
    x = Reshape((num_classes,))(x)
    #x = Dense(num_classes)(x)
    out = Activation('sigmoid')(x)

    model = Model(inputs = [inputs], outputs = out)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [20]:
model = define_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 398)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 398, 100)          596000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 398, 16)           7488      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 398, 11)           187       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 11)             48169     
_________________________________________________________________
reshape_1 (Reshape)          (None, 11)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 11)                0   

In [21]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20616 samples, validate on 5154 samples
Epoch 1/10
20616/20616 [==============================] - 31s 2ms/step - loss: 0.3503 - accuracy: 0.8965 - val_loss: 0.4599 - val_accuracy: 0.9091
Epoch 2/10
20616/20616 [==============================] - 29s 1ms/step - loss: 0.2961 - accuracy: 0.9091 - val_loss: 0.4544 - val_accuracy: 0.9091
Epoch 3/10
20616/20616 [==============================] - 29s 1ms/step - loss: 0.2952 - accuracy: 0.9091 - val_loss: 0.4543 - val_accuracy: 0.9091
Epoch 4/10
20616/20616 [==============================] - 29s 1ms/step - loss: 0.2949 - accuracy: 0.9091 - val_loss: 0.4404 - val_accuracy: 0.9091
Epoch 5/10
20616/20616 [==============================] - 31s 1ms/step - loss: 0.2941 - accuracy: 0.9091 - val_loss: 0.4531 - val_accuracy: 0.9091
Epoch 6/10
20616/20616 [==============================] - 29s 1ms/step - loss: 0.2923 - accuracy: 0.9091 - val_loss: 0.4499 - val_accuracy: 0.9091
Epoch 7/10
20616/20616 [==============================] - 29s 1ms/ste

In [22]:
model.evaluate(X_test, y_test)

1089/1089 [==============================] - 2s 2ms/step


[0.2833481353970161, 0.9075882434844971]

In [0]:
results = model.predict(X)

In [0]:
results_df = pd.DataFrame()
results_mask = results > 0.1
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [84]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

0

In [85]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,glam,modern,retro,romantic,attribute_value
0,androgynous,,,businesscasual,casual,classic,,,,,,"androgynous, businesscasual, casual, classic"
1,androgynous,,,businesscasual,casual,classic,,,,,,"androgynous, businesscasual, casual, classic"
2,,athleisure,,,casual,classic,,,modern,,,"athleisure, casual, classic, modern"
3,,,,businesscasual,casual,,,glam,modern,,,"businesscasual, casual, glam, modern"
4,,athleisure,,,casual,,,,,,,"athleisure, casual"


In [86]:
results_df['attribute_value'].value_counts()

androgynous, businesscasual, casual, classic                      2835
businesscasual, casual, classic, modern                            614
businesscasual, casual, classic                                    372
casual, classic, modern                                            330
androgynous, businesscasual, casual, classic, modern               280
                                                                  ... 
boho, businesscasual, casual, classic, modern, retro, romantic       2
androgynous, athleisure, businesscasual, edgy, glam, modern          2
androgynous, classic, edgy, glam, modern                             2
businesscasual, glam                                                 1
casual, classic, glam, romantic                                      1
Name: attribute_value, Length: 236, dtype: int64